In [1]:
import pandas as pd

In [2]:
def adjust_dataframe(dataframe):
    dataframe_ = dataframe[(dataframe['Year'] >= 1973)
                            &(dataframe['Geo Level'] == 'STATE')
                            &(dataframe['Data Item'] == 'CORN, GRAIN - YIELD, MEASURED IN BU / ACRE')]
    dataframe_ = dataframe_.drop(['Program', 'Week Ending', 'Ag District', 'Ag District Code',
                                 'County ANSI', 'State ANSI', 'Zip Code', 'Region',
                                 'watershed_code', 'Watershed', 'Commodity', 'Domain Category',
                                 'CV (%)', 'County', 'Geo Level', 'Domain', 'Data Item'], axis=1)
    dataframe_ = dataframe_[dataframe_['Period'] == 'YEAR']
    if dataframe_['Value'].dtype == 'object':
        dataframe_['Value'] = pd.to_numeric(dataframe_['Value'].str.replace(',', ''), errors='coerce')
    # Filtering relevant states
    relevant_states = ['IOWA','ILLINOIS','MINNESOTA','NEBRASKA', 'INDIANA','SOUTH DAKOTA',
                       'OHIO', 'WISCONSIN', 'KANSAS', 'MISSOURI','NORTH DAKOTA','MICHIGAN',
                       'KENTUCKY','TEXAS','ARKANSAS','COLORADO', 'PENNSYLVANIA','TENNESSEE']
    dataframe_ = dataframe_[dataframe_['State'].isin(relevant_states)]
    return dataframe_.drop(['Period'], axis=1).reset_index(drop=True)


In [3]:
data_path = '../../data/'
survey_yield = pd.read_csv(data_path + 'raw_data_USDA/survey_yield.csv')
sy = adjust_dataframe(survey_yield)
sy = sy.rename(columns={'Year':'ano'})


In [4]:
states_abbrv = {'AR': 'ARKANSAS', 'IA': 'IOWA', 'IL': 'ILLINOIS',
                'IN': 'INDIANA', 'KS': 'KANSAS', 'KY': 'KENTUCKY', 'MI': 'MICHIGAN',
                'MN': 'MINNESOTA', 'MO': 'MISSOURI', 'ND': 'NORTH DAKOTA',
                'NE': 'NEBRASKA', 'OH': 'OHIO', 'PA': 'PENNSYLVANIA',
                'SD': 'SOUTH DAKOTA', 'TN': 'TENNESSEE', 'WI': 'WISCONSIN', 'TX': 'TEXAS', 'CO': 'COLORADO'}

weathers = {}
for abbrv in states_abbrv:
    weathers[states_abbrv[abbrv]] = pd.read_csv(f'../../data/raw_data_NOAA/by_month_and_state/{abbrv}.csv')
weather_and_dev = {}
for state in states_abbrv.values():
    state_weather = weathers[state]
    merged = state_weather.merge(sy[sy['State'] == state], left_on='ano', right_on='ano')
    weather_and_dev[state] = merged

In [5]:
def pivot_weather_and_month(weather_data, dev_data):
    for variable in ['PRCP', 'TMean', 'TMAX', 'TMIN', 'diff']:
        for month in weather_data['mes'].unique():
            filtered = weather_data[weather_data['mes'] == month][['ano', variable]].rename({variable: 'mes_' + str(int(month)) + '_' + variable}, axis=1)
            dev_data = dev_data.merge(filtered, on='ano')
    return dev_data.sort_values('ano').reset_index(drop=True)

In [6]:
monthly_pivotted = {}
for state in states_abbrv.values():
    monthly_pivotted[state] = pivot_weather_and_month(weathers[state], sy[sy['State'] == state])

In [7]:
#transforming the data to a single dataframe
final_data = pd.concat(monthly_pivotted.values())
final_data.to_csv('../../data/processed_data/processed_data.csv', index=False)